In [29]:
# URL = 'https://arxiv.org/pdf/2403.18802.pdf'
URL = '/Users/shanekercheval/Downloads/ud923-rosenblum-garfinkel-paper.pdf'

# when extracting the text from the PDF, INCLUDE_AT and EXCLUDE_AT are used to determine where to
# start and stop extracting text. For example, if INCLUDE_AT is 'Abstract' and EXCLUDE_AT is
# 'Acknowledgements', then the text extraction will start at (and include) the first occurrence of
# 'Abstract' and stop at (and Exclude) the first occurrence of 'Acknowledgements'.
# INCLUDE_AT = "ABSTRACT"
INCLUDE_AT = "Virtual Machine"
# EXCLUDE_AT = "Acknowledgements"
EXCLUDE_AT = "References"

# MODEL = 'gpt-3.5-turbo-0125'
# MODEL = 'gpt-4-0125-preview'
MODEL = 'gpt-4o-mini'
SYSTEM_MESSAGE = 'You are an AI assistant that gives detailed and intuitive explanations.'
MAX_TOKENS=None
TEMPERATURE=0.8

---

In [37]:
from IPython.display import Markdown, display
from source.library.pdf import clean_text_from_pdf, extract_text_from_pdf
from llm_workflow.openai import OpenAIChat, num_tokens, MODEL_COST_PER_TOKEN


def create_model():
    return OpenAIChat(MODEL, SYSTEM_MESSAGE, max_tokens=MAX_TOKENS, temperature=TEMPERATURE)

In [40]:
# download and extract text of pdf
text = extract_text_from_pdf(pdf_path=URL)
n_tokens = num_tokens(model_name=MODEL, value=text + SYSTEM_MESSAGE)
print(f"# of tokens: {n_tokens:,}")
print(f"Cost if input tokens == {n_tokens:,}:  ${MODEL_COST_PER_TOKEN[MODEL]['input'] * n_tokens:.3f}")  # noqa: E501
print(f"Cost if output tokens == {n_tokens:,}: ${MODEL_COST_PER_TOKEN[MODEL]['output'] * n_tokens:.3f}")  # noqa: E501


# of tokens: 9,764
Cost if input tokens == 9,764:  $0.001
Cost if output tokens == 9,764: $0.006


In [41]:
# removed text before `INCLUDE_AT` and after `EXCLUDE_AT`
chars_before = len(text)
print(f"{chars_before:,} characters before")
text = clean_text_from_pdf(text=text, include_at=INCLUDE_AT, exclude_at=EXCLUDE_AT)
chars_after = len(text)
print(f"{chars_after:,} characters after")
print(f"Removed {abs((chars_after - chars_before) / chars_before):.2%} of text")
print("Preview:\n---\n")
print(text[:500])
print("\n...\n")
print(text[-500:])

42,707 characters before
40,184 characters after
Removed 5.91% of text
Preview:
---

Virtual Machine

Monitors: CurrentTechnology and Future Trends

A t the end of the 1960s, the virtual machine monitor (VMM) came into being as a soft- ware-abstraction layer that partitions a hardware platform into one or more virtual machines.1 Each of these virtual machines was sufﬁciently similar to the underlying physical machine to run existing software unmodiﬁed.

At the time, general-purpose computing was the domain of large, expensive mainframe hardware, and users found that VMMs provide

...

ndoning the strategy of procuring individual machines and tightly bundling complex software environments.

VMMs are giving these fragile, difﬁcult-to-manage systems new freedom. In coming years, virtual machines will move beyond their simple provi- sioning capabilities and beyond the machine room to provide a fundamental building block for mobility, security, and usability on the desktop.

Indeed, VMM cap

In [42]:
n_tokens = num_tokens(model_name=MODEL, value=text)
print(f"# of tokens: {n_tokens:,}")
print(f"Input cost if input tokens == {n_tokens:,}:   ${MODEL_COST_PER_TOKEN[MODEL]['input'] * n_tokens:.3f}")
print(f"Output cost if output tokens == {n_tokens:,}: ${MODEL_COST_PER_TOKEN[MODEL]['output'] * n_tokens:.3f}")

# of tokens: 7,793
Input cost if input tokens == 7,793:   $0.001
Output cost if output tokens == 7,793: $0.005


In [43]:
with open("source/library/prompts/summarize_pdf.txt") as f:
    summary_prompt = f.read()

model = create_model()
prompt = f"""
{summary_prompt}

Here is the paper:

{text}
"""

response = model(prompt)
with open('summary.txt', 'w') as f:
    f.write(response)
cost = model.cost
print(f"Cost: ${cost:.3f}")
display(Markdown(response))

Cost: $0.002


# Study Guide: Virtual Machine Monitors: Current Technology and Future Trends

## Virtual Machine Monitors: Current Technology and Future Trends

### Introduction to Virtual Machine Monitors (VMMs)

- **Virtual Machine Monitor (VMM)**: A software layer that partitions hardware into one or more virtual machines, each capable of running existing software without modification.
- The original use of VMMs emerged in the late 1960s, primarily for large, expensive mainframe computers. Users found VMMs useful for multiplexing these scarce resources across multiple applications.
- In the late 1980s and early 1990s, the rise of multitasking operating systems and decreased hardware costs diminished the necessity for VMMs, leading to their decline.
- The resurgence of VMMs around 2005 represents a renewed interest in their advantages, including enhanced mobility, security, and manageability in the face of evolving technological challenges.

### Why the Revival?

- VMMs are re-emerging due to:
  - The increase in low-cost hardware leading to underutilization of resources.
  - The complexity of modern operating systems, which have become fragile and vulnerable due to their increased capabilities.
- The VMM allows consolidation of multiple applications that previously ran on separate machines into fewer physical servers, thus optimizing resource use.
- **Server Consolidation**: The practice of running multiple virtual machines on fewer physical machines to reduce costs and improve efficiency.

### Decoupling Hardware and Software

- VMMs create a layer of indirection between virtual machines and the underlying hardware, which allows:
  - Control over how guest operating systems (GuestOSs) utilize hardware resources.
  - A uniform view of the hardware, enabling different machines to be treated as interchangeable.
- Key benefits of VMMs include:
  - Complete encapsulation of a virtual machine's software state, allowing for easy migration and load balancing.
  - Robust recovery options, such as suspending, checkpointing, and rolling back virtual machines.
  - Strong isolation between virtual machines, increasing reliability and security.

### VMM Implementation Issues

- Successful VMM design must balance:
  - **Compatibility**: The ability to run existing software without modification.
  - **Performance**: Minimizing virtualization overhead to ensure virtual machines run efficiently.
  - **Simplicity**: Reducing complexity to prevent failures that could affect multiple virtual machines.
- The VMM must effectively manage CPU, memory, and I/O virtualization to support these goals.

#### CPU Virtualization

- **CPU Virtualization**: The process by which the VMM allows multiple guest operating systems to run on a single CPU while maintaining control.
- Direct execution is the primary method, where the VMM executes virtual machines on real hardware while managing privileged operations.
- Challenges arise, particularly with architectures like x86, which require innovative solutions like paravirtualization or binary translation for effective virtualization.

#### Memory Virtualization

- **Memory Virtualization**: A technique where the VMM uses shadow page tables to manage memory allocations between virtual machines.
- Benefits include dynamic memory allocation and the ability to overcommit memory resources, allowing more virtual machines than physical memory would typically support.
- Challenges involve keeping shadow page tables updated and reducing redundant memory usage across multiple virtual instances.

#### I/O Virtualization

- **I/O Virtualization**: The management of input/output operations from virtual machines to physical devices.
- The evolution from IBM's channel-based architecture to modern x86 environments complicates I/O virtualization due to diverse device interfaces.
- VMware addresses I/O challenges with a hosted architecture that utilizes host operating system drivers, balancing performance and flexibility.

### What’s Ahead?

- The future of VMMs in server management will involve dynamic allocation of resources, automatic load balancing, and fast migration of virtual machines to ensure continuity and efficiency.
- On the desktop, VMMs can simplify the management of computing environments, allowing for greater mobility and user flexibility.
- The security potential of VMMs lies in their ability to isolate different software stacks and provide a previously unattainable level of assurance against attacks.
- Software distribution could also be revolutionized, with entire applications being packaged as virtual appliances, simplifying deployment and maintenance.

### Conclusion

- The resurgence of VMMs represents a paradigm shift in how software and hardware can be managed. As computing environments evolve, VMMs will play a crucial role in addressing future challenges related to mobility, security, and usability.
- The versatility and efficiency of VMMs will continue to shape the computing landscape, emphasizing their importance in the modern technological ecosystem. 

This study guide provides a structured overview of the key concepts and discussions within the paper, helping readers to understand the historical context, current trends, and future potential of Virtual Machine Monitors.

In [11]:
print(f"Total Cost:            ${model.cost:.5f}")
print(f"Total Tokens:          {model.total_tokens:,}")
print(f"Total Prompt Tokens:   {model.input_tokens:,}")
print(f"Total Response Tokens: {model.response_tokens:,}")

Total Cost:            $0.10213
Total Tokens:          8,753
Total Prompt Tokens:   8,023
Total Response Tokens: 730


---

In [ ]:
prompt = "Do the authors discuss the specifics or different scenarios of when not to use cosine similarity?"
response = model(prompt)
cost = model.cost
print(f"Cost: ${cost:.3f}")
display(Markdown(response))

---